In [2]:
import ee, math, datetime ,os
import numpy as np
import pandas as pd
import geopandas as gpd

In [3]:
from dotenv import load_dotenv

load_dotenv()
ee.Authenticate()
ee.Initialize(project=os.getenv('projectkey'))

In [ ]:
TARGET_CRS="EPSG:32736"
TARGET_SCALE=90#meters
RECEPTIVE_FIELD=15#pixels^2
rw_borders=ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('country_na','Rwanda')).geometry()
rw_bbox=ee.Geometry.BBox(28.8465, -2.8535, 30.9135, -1.0265) # 1.5 km padding around rwanda's borders
grid_fc=rw_bbox.coveringGrid(TARGET_CRS,TARGET_SCALE)

In [ ]:
noaa_gfs=ee.ImageCollection('NOAA/GFS0P25')
noaa_cfsv2=ee.ImageCollection("NOAA/CFSV2/FOR6H")
wwf_hdd=ee.Image("WWF/HydroSHEDS/03DIR").resample('bilinear').reproject(crs=TARGET_CRS,scale=TARGET_SCALE)
wwf_habl12=ee.FeatureCollection("WWF/HydroATLAS/v1/Basins/level12")

streams={
    noaa_gfs:['precipitation_rate','temperature_2m_above_ground'],
    noaa_cfsv2:'Volumetric_Soil_Moisture_Content_depth_below_surface_layer_5_cm',
    wwf_habl12:['for_pc_sse', 'kar_pc_sse', 'slp_dg_sav', 'wet_cl_smj', 'lka_pc_sse', 'ria_ha_ssu', 'lit_cl_smj', 'ero_kh_sav', 'ire_pc_sse', 'inu_pc_smx', 'dis_m3_pyr'],
    wwf_hdd:'b1'}

In [ ]:
# thanks for your time